## MODELO DE MACHINE LEARNING

In [1]:
import pandas as pd
df_movies = pd.read_csv('movies_ETL.csv')

Se desarrolló un script para implementar una función llamada `recomendacion` que genera recomendaciones de películas similares en función de un título de película de referencia.<br> A continuación veremos en detalle cómo funciona este script:

#### Paso 1:
**Preprocesamiento de datos:**

`df_movies_expanded = df_movies.explode('géneros')`: Esta línea crea un nuevo DataFrame llamado df_movies_expanded mediante la función explode, que expande los valores de la columna 'géneros' en varias filas. Esto permite que una película tenga múltiples filas, una por cada género al que pertenece.<br><br>
`df_movies_expanded['géneros'] = df_movies_expanded['géneros'].astype(str)`: Aquí, se convierten los valores de la columna 'géneros' en cadenas de texto para asegurarse de que sean del tipo correcto.<br><br>
`df_movies_expanded['popularity'] = df_movies_expanded['popularity'].astype(float)`: En esta línea, se convierten los valores de la columna 'popularity' a tipo de dato float para que puedan ser utilizados en cálculos posteriores.

#### Paso 2:
**Filtrar la película de referencia:**

`df_ref_movie = df_movies_expanded[df_movies_expanded['title'] == titulo]`: Se filtra el DataFrame df_movies_expanded para obtener la película de referencia con el título especificado. Esto crea un nuevo DataFrame llamado df_ref_movie que contiene la película de referencia.<br><br>
`ref_genres = set(df_ref_movie['géneros'])`: Se extraen los géneros de la película de referencia y se almacenan en un conjunto (set) llamado ref_genres. Esto se utilizará para calcular la similitud basada en los géneros de las películas.<br><br>
`ref_franchise = df_ref_movie['franquicia'].iloc[0]`: Se obtiene el nombre de la franquicia de la película de referencia y se almacena en la variable ref_franchise.<br><br>
`ref_actors = set(df_ref_movie['elenco'].apply(get_top_actors).iloc[0])`: Se extrae el elenco de la película de referencia y se aplica la función get_top_actors a cada valor del elenco para obtener los actores principales. Luego, se almacenan los actores principales en un conjunto llamado ref_actors. Esto se utilizará para calcular la similitud basada en el elenco de las películas.

#### Paso 3
**Calcular la puntuación de similitud:**

`df_movies_filtered = df_movies_expanded[df_movies_expanded['title'] != titulo]`: Se filtra el DataFrame df_movies_expanded para excluir la película de referencia y se crea un nuevo DataFrame llamado df_movies_filtered.<br><br>
`df_movies_filtered['similarity_score'] = ...`: En esta línea, se calcula la puntuación de similitud entre cada película y la película de referencia. Se realiza una serie de cálculos ponderados para diferentes características:<br><br>
-`df_movies_filtered['géneros'].apply(lambda genres: len(set(genres) & ref_genres)) * 15`: Se compara la columna 'géneros' de cada película con ref_genres para calcular la similitud basada en los géneros compartidos. Se asigna un peso de 15 a esta puntuación.<br>
-`(df_movies_filtered['franquicia'] == ref_franchise).astype(int) * 10`: Se compara la columna 'franquicia' de cada película con ref_franchise para verificar si comparten la misma franquicia. Se asigna un peso de 10 a esta puntuación.<br>
-`df_movies_filtered['popularity'] * 0.0001`: Se utiliza la columna 'popularity' como una medida de popularidad de la película. Se multiplica por un factor de 0.0001 para ajustar la escala y asignar una puntuación basada en la popularidad relativa.<br>
-`df_movies_filtered['title'].apply(lambda x: fuzz.ratio(x, titulo)) * 0.01`: Se utiliza la función fuzz.ratio de la biblioteca fuzzywuzzy para calcular la similitud entre el título de cada película y el título de referencia. Se multiplica por 0.01 para ajustar la escala y asignar una puntuación basada en la similitud del título.<br>
-`df_movies_filtered['elenco'].apply(lambda elenco: len(set(get_top_actors(elenco)) & ref_actors)) * 1`: Se aplica la función get_top_actors al elenco de cada película para obtener los actores principales y se compara con ref_actors para calcular la similitud basada en el elenco compartido. Se asigna un peso de 1 a esta puntuación.

#### Paso 4
**Ordenar las películas según la puntuación de similitud:**

`df_movies_sorted = df_movies_filtered.sort_values('similarity_score', ascending=False)`: Se ordena el DataFrame df_movies_filtered en orden descendente según la columna 'similarity_score'. Esto significa que las películas con una puntuación de similitud más alta aparecerán primero.

#### Paso 5
**Generar las recomendaciones:**

top_recommendations = df_movies_sorted['title'].head(5).tolist(): Se seleccionan los títulos de las primeras 5 películas del DataFrame ordenado y se almacenan en una lista llamada top_recommendations. Estas películas son las principales recomendaciones basadas en su similitud con la película de referencia.<br><br>
Finalmente, la función devuelve la lista top_recommendations, que contiene los títulos de las películas recomendadas.

En resumen, la función `recomendacion` utiliza múltiples características de las películas (géneros, franquicia, popularidad, título y elenco) para calcular una puntuación de similitud y generar una lista de recomendaciones basadas en esa similitud.

In [8]:
def recomendacion(titulo):
    # Preprocesamiento de datos
    df_movies_expanded = df_movies.explode('géneros')
    df_movies_expanded['géneros'] = df_movies_expanded['géneros'].astype(str)
    df_movies_expanded['popularity'] = df_movies_expanded['popularity'].astype(float)

    # Filtrar la película de referencia
    df_ref_movie = df_movies_expanded[df_movies_expanded['title'] == titulo]
    ref_genres = set(df_ref_movie['géneros'])
    ref_franchise = df_ref_movie['franquicia'].iloc[0]
    ref_actors = set(df_ref_movie['elenco'].apply(get_top_actors).iloc[0])

    # Calcular la puntuación de similitud
    df_movies_filtered = df_movies_expanded[df_movies_expanded['title'] != titulo]
    df_movies_filtered['similarity_score'] = (df_movies_filtered['géneros'].apply(lambda genres: len(set(genres) & ref_genres)) * 15 +
                                               (df_movies_filtered['franquicia'] == ref_franchise).astype(int) * 10 +
                                               (df_movies_filtered['popularity'] * 0.0001) +
                                               df_movies_filtered['title'].apply(lambda x: fuzz.ratio(x, titulo)) * 0.01 +
                                               df_movies_filtered['elenco'].apply(lambda elenco: len(set(get_top_actors(elenco)) & ref_actors)) * 1)

    # Ordenar las películas según la puntuación de similitud
    df_movies_sorted = df_movies_filtered.sort_values('similarity_score', ascending=False)

    # Generar las recomendaciones
    top_recommendations = df_movies_sorted['title'].head(5).tolist()

    return top_recommendations

In [9]:
resultado = recomendacion1("Star Wars")
print(resultado)  # Imprime "La productora Universal Pictures ha tenido una recaudación total de x, con un total de x películas producidas."

['Star Wars: The Force Awakens', 'Return of the Jedi', 'Star Wars: Episode I - The Phantom Menace', 'Star Wars: Episode III - Revenge of the Sith', 'Star Wars: Episode II - Attack of the Clones']


C:\Users\luise\AppData\Local\Temp\ipykernel_21860\2309790535.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_filtered['similarity_score'] = (df_movies_filtered['géneros'].apply(lambda genres: len(set(genres) & ref_genres)) +
